## Imports

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Instanciando o modelo

In [ ]:
sbert_model = SentenceTransformer('distiluse-base-multilingual-cased')
sbert_model

## Definindo caso de teste

In [ ]:
documents = ['Napoleão morreu de ataque cardíaco em setembro, e por isso ele perdeu a guerra',
'Napoles é uma cidade legal na itália onde napoleão passou férias',
'Napoleão morreu de infarto em setembro, e por isso ele ganhou a guerra', 
]

documents_df=pd.DataFrame(documents,columns=['documents'])

In [ ]:
list(documents_df['documents'])

## Codificando as frases

In [ ]:
document_embeddings = sbert_model.encode(documents_df['documents'])
document_embeddings

## Matriz de adjacencias ou matriz de similaridade

In [ ]:
sim_matrix = cosine_similarity(document_embeddings)
sim_matrix

## Selecionando os elementos acima da diagonal principal e filtrando

In [ ]:
sim_matrix = (sim_matrix > 0.5) * sim_matrix
array = []

for i in range(sim_matrix.shape[0]):
  for j in range(sim_matrix.shape[1]):
    if j > i:
      if sim_matrix[i][j] != 0.0:
        array.append({'pair': [i+1, j+1], 'similarity': sim_matrix[i][j]})

array